# Generate AMITT page metadata

Create a page for each of the AMITT objects, if it doesn't already exist.  If it does, update the metadata on it, but take care to preserve any human-generated notes. 

In [1]:
# Libraries and functions
import pandas as pd
import numpy as np
import os

def create_incidentstring(incidentlist):

    incidentstr = '''
| Incident |
| --------- |
'''
    incirow = '| [{0} {1}](../incidents/{0}.md) |\n'
    for index, row in incidentlist[['id_incident', 'name_incident']].drop_duplicates().sort_values('id_incident').iterrows():
        incidentstr += incirow.format(row['id_incident'], row['name_incident'])
    return incidentstr


def create_techstring(techlist):

    techstr = '''
| Technique | Description given for this incident |
| --------- | ------------------------- |
'''

    techrow = '| [{0} {1}](../techniques/{0}.md) | {2} {3} |\n'
    for index, row in techlist.sort_values('id_technique').iterrows():
        techstr += techrow.format(row['id_technique'], row['name_technique'], 
                                  row['id'], row['name'])
    return techstr


# Load metadata from file
xlsx = pd.ExcelFile('amitt_metadata_v3.xlsx')
metadata = {}
for sheetname in xlsx.sheet_names:
    metadata[sheetname] = xlsx.parse(sheetname)
    
# Generate full cross-table between incidents and techniques
it = metadata['incidenttechniques']
it.index=it['id']
it = it['techniques'].str.split(',').apply(lambda x: pd.Series(x)).stack().reset_index(level=1, drop=True).to_frame('technique').reset_index().merge(it.drop('id', axis=1).reset_index()).drop('techniques', axis=1)
it = it.merge(metadata['incidents'][['id','name']], 
              left_on='incident', right_on='id',
              suffixes=['','_incident']).drop('incident', axis=1)
it = it.merge(metadata['techniques'][['id','name']], 
              left_on='technique', right_on='id',
              suffixes=['','_technique']).drop('technique', axis=1)

print('loaded {}'.format(list(metadata.keys())))

loaded ['phases', 'tactics', 'techniques', 'tasks', 'incidents', 'incidenttechniques']


In [2]:
# Generate datafiles
warntext = 'DO NOT EDIT ABOVE THIS LINE - PLEASE ADD NOTES BELOW'
warnlen = len(warntext)

for entity in ['phase', 'tactic', 'technique', 'task', 'incident']:
    entities = entity + 's'
    entitydir = '../{}'.format(entities)
    if not os.path.exists(entitydir):
        os.makedirs(entitydir)
    
    template = open('template_{}.md'.format(entity)).read()
    dfentity = metadata[entities]
    for index, row in dfentity[dfentity['name'].notnull()].iterrows():
        
        # First read in the file - if it exists - and grab everything 
        # below the "do not write about this line". Will write this 
        # out below new metadata. 
        datafile = '../{}/{}.md'.format(entities, row['id'])
        if os.path.exists(datafile):
            with open(datafile) as f:
                filetext = f.read()
            warnpos = filetext.find(warntext)
            if warnpos == -1:
                print('no warning text found in {}: not updating file'.format(datafile))
                continue
            usertext = filetext[warnpos+warnlen:]
        else:
            usertext = ''
        
        # Now populate datafiles with new metadata plus old userdata
        with open(datafile, 'w') as f:
            if entity == 'phase':
                f.write(template.format(id=row['id'], name=row['name'],
                                        summary=row['summary']))
            if entity == 'tactic':
                f.write(template.format(id=row['id'], name=row['name'],
                                        phase=row['phase'], summary=row['summary']))            
            if entity == 'task':
                f.write(template.format(id=row['id'], name=row['name'],
                                        tactic=row['tactic'], summary=row['summary']))
            if entity == 'technique':
                f.write(template.format(id=row['id'], name=row['name'],
                                        tactic=row['tactic'], summary=row['summary'],
                                        incidents=create_incidentstring(it[it['id_technique'] == row['id']])))
            if entity == 'incident':
                f.write(template.format(id=row['id'], name=row['name'],
                                        type=row['type'], summary=row['summary'],
                                        yearstarted=row['Year Started'], 
                                        fromcountry=row['From country'],
                                        tocountry=row['To country'],
                                        foundvia=row['Found via'],
                                        dateadded=row['When added'],
                                        techniques=create_techstring(it[it['id_incident'] == row['id']])))

            # Make sure the user data goes in
            f.write(warntext)
            f.write(usertext)
            f.close()

In [3]:
for technique in metadata['techniques']['id'].to_list():
    print('{}'.format(create_incidentstring(it[it['id_technique'] == technique])))


| Incident |
| --------- |
| [I00029 MH17 investigation](../incidents/I00029.md) |
| [I00047 Sea of Azov](../incidents/I00047.md) |
| [I00049 White Helmets: Chemical Weapons](../incidents/I00049.md) |
| [I00053 China Huawei CFO Arrest](../incidents/I00053.md) |
| [I00063 Olympic Doping Scandal](../incidents/I00063.md) |


| Incident |
| --------- |
| [I00033 China 50cent Army](../incidents/I00033.md) |
| [I00034 DibaFacebookExpedition](../incidents/I00034.md) |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |
| [I00005 Brexit vote](../incidents/I00005.md) |
| [I00006 Columbian Chemicals](../incidents/I00006.md) |
| [I00009 PhilippinesExpert](../incidents/I00009.md) |
| [I00010 ParklandTeens](../incidents/I00010.md) |
| [I00017 US presidential elections](../incidents/I00017.md) |
| [I00022 #Macronleaks](../incidents/I00022.md) |
| [I00032 Kavanaugh](../incidents/I00032.md) |
| [I00056 Iran I


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |
| --------- |


| Incident |


In [4]:
for incident in metadata['incidents']['id'].to_list():
    print('{}'.format(create_techstring(it[it['id_incident'] == incident])))


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |
| [T0017 Promote online funding](../techniques/T0017.md) | I00002T002 Promote "funding" campaign |
| [T0018 Paid targeted ads](../techniques/T0018.md) | I00002T001 buy FB targeted ads |
| [T0019 Generate information pollution](../techniques/T0019.md) | I00002T003 create web-site - information pollution |
| [T0046 Search Engine Optimization](../techniques/T0046.md) | I00002T004 SEO optimisation/manipulation ("key words") |
| [T0056 Dedicated channels disseminate information pollution](../techniques/T0056.md) | I00002T003 create web-site - information pollution |
| [T0058 Legacy web content](../techniques/T0058.md) | I00002T005 legacy web content |
| [T0058 Legacy web content](../techniques/T0058.md) | I00002T006 hard to remove content and/or campaign/exploit TOS |


| Technique | Description given f


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |
| [T0001 4Ds](../techniques/T0001.md) | I00029T003 4 D's (dismiss, distort, distract, dismay...Official channels ie: Embassies & Defence Ministry; TASS |
| [T0010 Cultivate useful idiots](../techniques/T0010.md) | I00029T005 cultivate, manipulate, exploit useful idiots |
| [T0019 Generate information pollution](../techniques/T0019.md) | I00029T004 RT & Sputnik generate information pollution (synthetic media) |
| [T0031 YouTube](../techniques/T0031.md) | I00029T008 YouTube; Reddit;  |
| [T0032 Reddit](../techniques/T


| Technique | Description given for this incident |
| --------- | ------------------------- |
| [T0001 4Ds](../techniques/T0001.md) | I00049T003 4 D's (dismiss, distort, distract, dismay...Official channels ie: Embassies & Defence Ministry; TASS |
| [T0010 Cultivate useful idiots](../techniques/T0010.md) | I00049T005 cultivate, manipulate, exploit useful idiots (Roger Waters; Venessa Beeley...) |
| [T0019 Generate information pollution](../techniques/T0019.md) | I00049T004 RT & Sputnik generate information pollution (synthetic media) |
| [T0031 YouTube](../techniques/T0031.md) | I00049T008 YouTube; Reddit;  |
| [T0032 Reddit](../techniques/T0032.md) | I00049T008 YouTube; Reddit;  |
| [T0046 Search Engine Optimization](../techniques/T0046.md) | I00049T006 SEO optimisation/manipulation ("key words") |
| [T0052 Tertiary sites amplify news](../techniques/T0052.md) | I00049T007 News circulated/amplifed by tertiary sites (Russia Insider, The Duran, Geopolitica.ru, Mint Press News, Oriental 



| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for this incident |
| --------- | ------------------------- |


| Technique | Description given for th

In [5]:
# todo: create the matrix (clickable?)
# todo: add in the existing incident pages
# todo: add in the existing tactic notes from spreadsheets
# todo: add all framework comments to the repo issues list

## Make the grid

In [6]:
# Create the master grid that we make all the framework visuals from
# cols = number of tactics
# rows = max number of techniques per tactic + 2
phases = metadata['phases']
phasedict = pd.Series(phases.name.values,index=phases.id).to_dict()

techniques = metadata['techniques']
tactechs = techniques.groupby('tactic')['id'].apply(list).reset_index().rename({'id':'techniques'}, axis=1)
maxtechs = max(tactechs['techniques'].apply(len))
techdict = pd.Series(techniques.name.values,index=techniques.id).to_dict()

tactics = metadata['tactics'].merge(tactechs, left_on='id', right_on='tactic', how='left').fillna('').drop('tactic', axis=1)
tacdict = pd.Series(tactics.name.values,index=tactics.id).to_dict()


rows, cols = (maxtechs+2, len(tactics))
arr = [['' for i in range(cols)] for j in range(rows)] 
for index, tactic in tactics.iterrows():
    arr[0][index] = tactic['phase']
    arr[1][index] = tactic['id']
    if tactic['techniques'] == '':
        continue
    for index2, technique in enumerate(tactic['techniques']):
        arr[index2+2][index] = technique

#Save grid to file
matrixdir = '../matrices'
if not os.path.exists(matrixdir):
    os.makedirs(matrixdir)
pd.DataFrame(arr).to_csv(matrixdir + '/matrix_arr.csv', index=False, header=False)

arr

[['P01',
  'P01',
  'P02',
  'P02',
  'P02',
  'P02',
  'P02',
  'P03',
  'P03',
  'P03',
  'P03',
  'P04'],
 ['TA01',
  'TA02',
  'TA03',
  'TA04',
  'TA05',
  'TA06',
  'TA07',
  'TA08',
  'TA09',
  'TA10',
  'TA11',
  'TA12'],
 ['T0001',
  'T0005',
  'T0007',
  'T0010',
  'T0016',
  'T0019',
  'T0029',
  'T0039',
  'T0047',
  'T0057',
  'T0058',
  ''],
 ['T0002',
  'T0006',
  'T0008',
  'T0011',
  'T0017',
  'T0020',
  'T0030',
  'T0040',
  'T0048',
  '',
  'T0059',
  ''],
 ['T0003',
  '',
  'T0009',
  'T0012',
  'T0018',
  'T0021',
  'T0031',
  'T0041',
  'T0049',
  '',
  'T0060',
  ''],
 ['T0004',
  '',
  '',
  'T0013',
  '',
  'T0022',
  'T0032',
  'T0042',
  'T0050',
  '',
  '',
  ''],
 ['', '', '', 'T0014', '', 'T0023', 'T0033', 'T0043', 'T0051', '', '', ''],
 ['', '', '', 'T0015', '', 'T0024', 'T0034', 'T0044', 'T0052', '', '', ''],
 ['', '', '', '', '', 'T0025', 'T0035', 'T0045', 'T0053', '', '', ''],
 ['', '', '', '', '', 'T0026', 'T0036', 'T0046', 'T0054', '', '', ''],
 [''

In [7]:
# Write HTML version of framework diagram

html = '''# AMITT Latest Framework:

<table border="1">
<tr>
'''

for col in range(cols):
    html += '<td><a href="phases/{0}.md">{0} {1}</a></td>'.format(
        arr[0][col], phasedict[arr[0][col]])
html += '</tr>\n<tr>'

for col in range(cols):
    html += '<td><a href="tactics/{0}.md">{0} {1}</a></td>'.format(
        arr[1][col], tacdict[arr[1][col]])
html += '</tr>\n<tr>'

for row in range(2,rows):
    for col in range(cols):
        if arr[row][col] == '':
            html += '<td> </td>'
        else:
            html += '<td><a href="techniques/{0}.md">{0} {1}</a></td>'.format(
                arr[row][col], techdict[arr[row][col]])
    html += '</tr>\n<tr>'
html += '</tr></table>'
with open('../matrix.md', 'w') as f:
    f.write(html)

In [8]:
# Write HTML version of incident list

html = '''# AMITT Incidents:

<table border="1">
<tr>
'''

cols = ['name', 'type', 'Year Started', 'From country', 'To country',
        'Found via']

html += '<th>{}</th>'.format('id')
for col in cols:
    html += '<th>{}</th>'.format(col)
html += '</tr>\n'

incidents = metadata['incidents']
for index, row in incidents[incidents['name'].notnull()].iterrows():
    html += '\n<tr>'
    html += '<td><a href="incidents/{0}.md">{0}</a></td>'.format(row['id'])
    for col in cols:
            html += '<td>{}</td>'.format(row[col])
    html += '</tr>\n'
html += '</table>'
with open('../incidents.md', 'w') as f:
    f.write(html)